## Creating the AWS Serverless Application Model (SAM) application

### Verify pre-requisites for AWS SAM
SAM requires python 2.7 or 3.6 and pip to be installed to continue.

In [ ]:
import os
import sagemaker
from sagemaker import get_execution_role
# import project_path
# from lib import utils
import boto3

role = get_execution_role()

# use the region-specific sample data bucket
region = boto3.Session().region_name
bucket = '{{s3_workshop_bucket}}'
stack_name = "serverless-hello"

In [ ]:
!python --version
!pip --version

### Installing AWS SAM

In [ ]:
!pip install --user aws-sam-cli

### Verify the latest version of SAM is installed 

In [ ]:
!python -m site --user-base

In [ ]:
%%bash
export PATH=$PATH:/home/ec2-user/.local/bin
sam --version

In [ ]:
%bash
/home/ec2-user/.local/bin/sam init -r python3.6 -n serverless-todo

In [ ]:
!ls serverless-todo/

In [ ]:
!cat serverless-todo/template.yaml

In [ ]:
cd serverless-todo 

In [ ]:
!pip install -r hello_world/requirements.txt -t hello_world/

In [ ]:
!ls hello_world

In [ ]:
!pygmentize hello_world/app.py

In [ ]:
!aws cloudformation validate-template --template-body file://template.yaml

In [ ]:
!aws cloudformation package \
    --template-file template.yaml \
    --output-template-file sam-template.yaml \
    --s3-bucket analytics-serverless-west \
    --s3-prefix serverless

In [ ]:
!aws cloudformation deploy \
    --template-file sam-template.yaml \
    --stack-name serverless-hello \
    --capabilities CAPABILITY_IAM

In [ ]:
# api_endpoint = !aws cloudformation describe-stacks --stack-name $stack_name --region $region | python -c 'import json,sys;obj=json.load(sys.stdin);print obj["Stacks"][0]["Outputs"][1]["OutputValue"];'
# print(api_endpoint[0])

!aws cloudformation describe-stacks --stack-name $stack_name --region $region

After we create and deploy the project we will install httpie to test the endpoint to exercise the default method.

In [ ]:
!pip install httpie
!http https://ud6tcavio8.execute-api.us-west-2.amazonaws.com/Prod/hello/

### We will now make changes to the hello world project and create a todo application that leverages Cognito for user authentication.

We will start with creating the CRUD methods for the todo app to create, read, update, delete, and list the todos. We will also be using a DynamoDB table to store the todos. 

In your project copy the todos folder over to your project and update the serverless.yml file with the new API endpoints.

```
create:
    handler: todos/create.create
    events:
      - http:
          path: todos
          method: post
          cors: true

  list:
    handler: todos/list.list
    events:
      - http:
          path: todos
          method: get
          cors: true

  get:
    handler: todos/get.get
    events:
      - http:
          path: todos/{id}
          method: get
          cors: true

  update:
    handler: todos/update.update
    events:
      - http:
          path: todos/{id}
          method: put
          cors: true

  delete:
    handler: todos/delete.delete
    events:
      - http:
          path: todos/{id}
          method: delete
          cors: true
```

Once you have copied the todo Lambda functions into your project and updated the serverless.yml file we are ready to redeploy the project with the updated methods and test them out.

In [ ]:
!sls deploy -v

### Create a todo

In [ ]:
echo '{ "text": "My First Todo" }' | http POST https://{api-url}/dev/todos
http https://{api-url}/dev/todos

### List the todos available

In [ ]:
!http POST https://{api-url}/dev/todos

Now you can see how easy it is to create a microservice using Lambda but what if we wanted to only allow authenticated users to have the ability to create todos? We can create a Cognito user pool and use it as an authorizer to the API. The deployment should have already created the Cognito user pool, so we will use that and hook it up to the create method of the service. 

Change the serverless.yml file to add the authorizer like below.

```
    authorizer:
        type: COGNITO_USER_POOLS
        authorizerId:
            Ref: TodoApiGatewayAuthorizer
```

Let's redeploy the service and try and add a todo

In [ ]:
!sls deploy -v

In [ ]:
!echo '{ "text": "My First Todo" }' | http POST https://{api-url}/dev/todos

This should have returned an HTTP/1.1 401 Unauthorized because you did not send the appropriate Authorization header on the call. We will simulate a user logging into the application and passing the bearer token in the headers of the request. Running the 3 methods below will simulate creating a user, confirming the user, and generating the appropriate auth for the user to be able to create the todo. Grab the IdToken generated and add it to the headers like the command below.

### Create user in Cognito

In [ ]:
!aws cognito-idp sign-up --region {your-aws-region} --client-id {cognito-client-id} --username admin@example.com --password {password}

### Confirm the sign up of the user

In [ ]:
!aws cognito-idp admin-confirm-sign-up --region {your-aws-region} --user-pool-id {cognito-user-pool} --username admin@example.com

### Initiate the authentication to generate the token

In [ ]:
!aws cognito-idp admin-initiate-auth --region {your-aws-region} --cli-input-json file://auth.json

In [ ]:
!echo '{ "text": "My Authenticated Todo" }' | http POST https://{api-url}/dev/todos Authorization:"Bearer {your-idtoken}"

Now you should have created a new todo after the authorizer approves the token.

### Register a user

> aws cognito-idp sign-up --region {your-aws-region} --client-id {your-client-id} --username admin@example.com --password {password}

### Confirm user registration

> aws cognito-idp admin-confirm-sign-up --region {your-aws-region} --user-pool-id {your-user-pool-id} --username admin@example.com

### Authenticate (get tokens)

> aws cognito-idp admin-initiate-auth --region {your-aws-region} --cli-input-json file://auth.json

### Where auth.json is:

>{
    "UserPoolId": "{your-user-pool-id}",
    "ClientId": "{your-client-id}",
    "AuthFlow": "ADMIN_NO_SRP_AUTH",
    "AuthParameters": {
        "USERNAME": "admin@example.com",
        "PASSWORD": "{password}"
    }
}

### You should get a response like this if everything is set up correctly:

>{
    "AuthenticationResult": {
        "ExpiresIn": 3600,
        "IdToken": "{your-idtoken}",
        "RefreshToken": "{your-refresh-token}",
        "TokenType": "Bearer",
        "AccessToken": "{your-access-token}"
    },
    "ChallengeParameters": {}
}

## Cleanup

In [ ]:
!aws cloudformation delete-stack --stack-name $stack_name